In [90]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from datasets import load_dataset
import warnings
import pandas as pd
warnings.filterwarnings('ignore')
from cnn import CNNFromScratch, build_cnn_from_keras, test_implementation_consistency, calculate_macro_f1_score
from rnn import RNNModelFromScratch, train_keras_model as train_rnn_model, compare_implementations as compare_rnn
from lstm import LSTMFromScratch, create_lstm_model, train_model as train_lstm_model, compare_implementations as compare_lstm

In [91]:
# load cifar
(x_train_full, y_train_full), (x_test, y_test) = cifar10.load_data()
x_train_full = x_train_full.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train_full = y_train_full.flatten()
y_test = y_test.flatten()

In [92]:
# split data
x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_train_full,test_size=0.2, random_state=42)

In [93]:
def create_cnn_model(conv_layers=3, filters_list=[32, 64, 128], filter_sizes=[(3,3), (3,3), (3,3)], pooling='max'):
    model = keras.Sequential([
        layers.Input(shape=(32, 32, 3))
    ])
    
    for i in range(conv_layers):
        filters = filters_list[i] if i < len(filters_list) else filters_list[-1]
        kernel = filter_sizes[i] if i < len(filter_sizes) else filter_sizes[-1]
        
        model.add(layers.Conv2D(filters, kernel, activation='relu', padding='same'))
        if pooling == 'max':
            model.add(layers.MaxPooling2D((2, 2)))
        else:
            model.add(layers.AveragePooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [94]:
def train_cnn(model, epochs=5):
    history = model.fit(x_train, y_train, batch_size=32, epochs=epochs, validation_data=(x_val, y_val), verbose=1)
    pred = model.predict(x_test)
    pred_classes = np.argmax(pred, axis=1)
    f1 = calculate_macro_f1_score(y_test, pred_classes)
    return history, f1

In [95]:
# pengaruh jumlah layer
print("Testing: Jumlah Layer Konvolusi")
layer_results = {}

for num_layers in [2, 3, 4]:
    print(f"\nTraining dengan {num_layers} layers...")
    model = create_cnn_model(conv_layers=num_layers)
    history, f1 = train_cnn(model)
    layer_results[num_layers] = {'f1': f1, 'history': history}
    model.save(f'test_result/cnn/cnn_{num_layers}_layers.h5')
    print(f"F1-Score: {f1:.4f}")

Testing: Jumlah Layer Konvolusi

Training dengan 2 layers...
Epoch 1/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 18s 14ms/step - accuracy: 0.4061 - loss: 1.6474 - val_accuracy: 0.6062 - val_loss: 1.1344
Epoch 2/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 16s 13ms/step - accuracy: 0.6205 - loss: 1.0756 - val_accuracy: 0.6573 - val_loss: 0.9815
Epoch 3/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 16s 13ms/step - accuracy: 0.6874 - loss: 0.8981 - val_accuracy: 0.6648 - val_loss: 0.9545
Epoch 4/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 16s 13ms/step - accuracy: 0.7278 - loss: 0.7844 - val_accuracy: 0.6925 - val_loss: 0.8831
Epoch 5/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - accuracy: 0.7603 - loss: 0.6790 - val_accuracy: 0.6880 - val_loss: 0.9089
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


F1-Score: 0.6816

Training dengan 3 layers...
Epoch 1/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 23s 17ms/step - accuracy: 0.3780 - loss: 1.6911 - val_accuracy: 0.5875 - val_loss: 1.1582
Epoch 2/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - accuracy: 0.6318 - loss: 1.0501 - val_accuracy: 0.6805 - val_loss: 0.9176
Epoch 3/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - accuracy: 0.7024 - loss: 0.8486 - val_accuracy: 0.7094 - val_loss: 0.8269
Epoch 4/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - accuracy: 0.7516 - loss: 0.7136 - val_accuracy: 0.7209 - val_loss: 0.8161
Epoch 5/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 18s 15ms/step - accuracy: 0.7884 - loss: 0.6073 - val_accuracy: 0.7292 - val_loss: 0.7737
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


F1-Score: 0.7238

Training dengan 4 layers...
Epoch 1/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 25s 19ms/step - accuracy: 0.3370 - loss: 1.7837 - val_accuracy: 0.5587 - val_loss: 1.2073
Epoch 2/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 24s 19ms/step - accuracy: 0.6020 - loss: 1.1134 - val_accuracy: 0.6351 - val_loss: 1.0284
Epoch 3/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 23s 18ms/step - accuracy: 0.6858 - loss: 0.8833 - val_accuracy: 0.6943 - val_loss: 0.8684
Epoch 4/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 23s 19ms/step - accuracy: 0.7379 - loss: 0.7458 - val_accuracy: 0.7253 - val_loss: 0.8052
Epoch 5/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 21s 17ms/step - accuracy: 0.7727 - loss: 0.6442 - val_accuracy: 0.7137 - val_loss: 0.8579
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step


F1-Score: 0.6924


In [96]:
# variasi jumlah filter
print("\nTesting: Jumlah Filter per Layer")
filter_results = {}
filter_configs = [[16, 32, 64], [32, 64, 128], [64, 128, 256]]

for i, filters in enumerate(filter_configs):
    print(f"\nTraining dengan filters: {filters}...")
    model = create_cnn_model(filters_list=filters)
    history, f1 = train_cnn(model)
    filter_results[str(filters)] = {'f1': f1, 'history': history}
    model.save(f'test_result/cnn/cnn_filters_{i}.h5')
    print(f"F1-Score: {f1:.4f}")


Testing: Jumlah Filter per Layer

Training dengan filters: [16, 32, 64]...
Epoch 1/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.3694 - loss: 1.7104 - val_accuracy: 0.5921 - val_loss: 1.1594
Epoch 2/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.6106 - loss: 1.1125 - val_accuracy: 0.6258 - val_loss: 1.0670
Epoch 3/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.6725 - loss: 0.9349 - val_accuracy: 0.6624 - val_loss: 0.9585
Epoch 4/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.7165 - loss: 0.8156 - val_accuracy: 0.6916 - val_loss: 0.8675
Epoch 5/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.7392 - loss: 0.7420 - val_accuracy: 0.6984 - val_loss: 0.8656
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


F1-Score: 0.6919

Training dengan filters: [32, 64, 128]...
Epoch 1/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - accuracy: 0.3754 - loss: 1.6992 - val_accuracy: 0.6069 - val_loss: 1.1043
Epoch 2/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - accuracy: 0.6352 - loss: 1.0300 - val_accuracy: 0.6778 - val_loss: 0.9220
Epoch 3/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - accuracy: 0.7097 - loss: 0.8253 - val_accuracy: 0.6937 - val_loss: 0.8828
Epoch 4/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - accuracy: 0.7577 - loss: 0.6970 - val_accuracy: 0.7331 - val_loss: 0.7724
Epoch 5/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - accuracy: 0.7911 - loss: 0.6027 - val_accuracy: 0.7495 - val_loss: 0.7556
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


F1-Score: 0.7388

Training dengan filters: [64, 128, 256]...
Epoch 1/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 50s 40ms/step - accuracy: 0.3551 - loss: 1.7504 - val_accuracy: 0.5771 - val_loss: 1.1778
Epoch 2/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 50s 40ms/step - accuracy: 0.6274 - loss: 1.0616 - val_accuracy: 0.6781 - val_loss: 0.9203
Epoch 3/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 54s 43ms/step - accuracy: 0.7067 - loss: 0.8343 - val_accuracy: 0.7040 - val_loss: 0.8408
Epoch 4/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 53s 42ms/step - accuracy: 0.7566 - loss: 0.6985 - val_accuracy: 0.7238 - val_loss: 0.8009
Epoch 5/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 52s 42ms/step - accuracy: 0.7970 - loss: 0.5825 - val_accuracy: 0.7380 - val_loss: 0.7913
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step


F1-Score: 0.7323


In [97]:
# variasi ukuran filter
print("\nTesting: Ukuran Filter")
size_results = {}

size_configs = [[(3,3), (3,3), (3,3)], [(5,5), (3,3), (3,3)], [(7,7), (5,5), (3,3)]]

for i, sizes in enumerate(size_configs):
    print(f"\nTraining dengan sizes: {sizes}...")
    model = create_cnn_model(filter_sizes=sizes)
    history, f1 = train_cnn(model)
    size_results[str(sizes)] = {'f1': f1, 'history': history}
    model.save(f'cnn_sizes_{i}.h5')
    print(f"F1-Score: {f1:.4f}")


Testing: Ukuran Filter

Training dengan sizes: [(3, 3), (3, 3), (3, 3)]...
Epoch 1/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 21s 16ms/step - accuracy: 0.3688 - loss: 1.7245 - val_accuracy: 0.6062 - val_loss: 1.1093
Epoch 2/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - accuracy: 0.6210 - loss: 1.0720 - val_accuracy: 0.6465 - val_loss: 1.0079
Epoch 3/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - accuracy: 0.6991 - loss: 0.8646 - val_accuracy: 0.6785 - val_loss: 0.9172
Epoch 4/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 21s 17ms/step - accuracy: 0.7448 - loss: 0.7308 - val_accuracy: 0.7153 - val_loss: 0.8242
Epoch 5/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 21s 17ms/step - accuracy: 0.7831 - loss: 0.6201 - val_accuracy: 0.7198 - val_loss: 0.8501
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step


F1-Score: 0.7123

Training dengan sizes: [(5, 5), (3, 3), (3, 3)]...
Epoch 1/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 29s 22ms/step - accuracy: 0.3631 - loss: 1.7384 - val_accuracy: 0.5603 - val_loss: 1.2249
Epoch 2/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 22s 18ms/step - accuracy: 0.5942 - loss: 1.1370 - val_accuracy: 0.6388 - val_loss: 1.0087
Epoch 3/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 21s 17ms/step - accuracy: 0.6770 - loss: 0.9167 - val_accuracy: 0.6571 - val_loss: 0.9999
Epoch 4/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 22s 17ms/step - accuracy: 0.7309 - loss: 0.7642 - val_accuracy: 0.6972 - val_loss: 0.8810
Epoch 5/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 22s 17ms/step - accuracy: 0.7676 - loss: 0.6638 - val_accuracy: 0.6997 - val_loss: 0.8746
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step


F1-Score: 0.6921

Training dengan sizes: [(7, 7), (5, 5), (3, 3)]...
Epoch 1/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 36s 28ms/step - accuracy: 0.3363 - loss: 1.7978 - val_accuracy: 0.5164 - val_loss: 1.3538
Epoch 2/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 36s 28ms/step - accuracy: 0.5505 - loss: 1.2600 - val_accuracy: 0.6079 - val_loss: 1.1229
Epoch 3/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 36s 28ms/step - accuracy: 0.6379 - loss: 1.0280 - val_accuracy: 0.6389 - val_loss: 1.0438
Epoch 4/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 36s 29ms/step - accuracy: 0.6894 - loss: 0.8802 - val_accuracy: 0.6519 - val_loss: 1.0141
Epoch 5/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 36s 29ms/step - accuracy: 0.7417 - loss: 0.7424 - val_accuracy: 0.6681 - val_loss: 1.0014
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step


F1-Score: 0.6636


In [98]:
# variasi jenis pooling

print("\nTesting: Jenis Pooling")
pooling_results = {}

for pooling in ['max', 'average']:
    print(f"\nTraining dengan {pooling} pooling...")
    model = create_cnn_model(pooling=pooling)
    history, f1 = train_cnn(model)
    pooling_results[pooling] = {'f1': f1, 'history': history}
    model.save(f'test_result/cnn/cnn_{pooling}_pooling.h5')
    print(f"F1-Score: {f1:.4f}")


Testing: Jenis Pooling

Training dengan max pooling...
Epoch 1/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 22s 17ms/step - accuracy: 0.3754 - loss: 1.6990 - val_accuracy: 0.5854 - val_loss: 1.1719
Epoch 2/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 21s 17ms/step - accuracy: 0.6194 - loss: 1.0827 - val_accuracy: 0.6608 - val_loss: 0.9471
Epoch 3/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 22s 18ms/step - accuracy: 0.6894 - loss: 0.8829 - val_accuracy: 0.6930 - val_loss: 0.8818
Epoch 4/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 21s 17ms/step - accuracy: 0.7406 - loss: 0.7431 - val_accuracy: 0.6825 - val_loss: 0.9151
Epoch 5/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 21s 17ms/step - accuracy: 0.7771 - loss: 0.6368 - val_accuracy: 0.7015 - val_loss: 0.8825
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step


F1-Score: 0.6912

Training dengan average pooling...
Epoch 1/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - accuracy: 0.3538 - loss: 1.7605 - val_accuracy: 0.5540 - val_loss: 1.2502
Epoch 2/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - accuracy: 0.5691 - loss: 1.2043 - val_accuracy: 0.6299 - val_loss: 1.0509
Epoch 3/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - accuracy: 0.6504 - loss: 1.0009 - val_accuracy: 0.6743 - val_loss: 0.9298
Epoch 4/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - accuracy: 0.6899 - loss: 0.8786 - val_accuracy: 0.6844 - val_loss: 0.8891
Epoch 5/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 19s 16ms/step - accuracy: 0.7302 - loss: 0.7652 - val_accuracy: 0.7225 - val_loss: 0.8034
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step


F1-Score: 0.7096


In [99]:
# CNN From Scratch Testing
print("\nTesting CNN From Scratch Implementation")
best_model_path = 'test_result/cnn/cnn_3_layers.h5'

test_results = test_implementation_consistency(best_model_path, (x_test, y_test), num_samples=100)

print(f"Keras F1: {test_results['keras_f1']:.4f}")
print(f"Scratch F1: {test_results['scratch_f1']:.4f}")
print(f"Implementation Accuracy: {test_results['implementation_accuracy']:.4f}")


Testing CNN From Scratch Implementation


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Keras F1: 0.7225
Scratch F1: 0.7225
Implementation Accuracy: 1.0000


In [100]:
# load data
train_data = pd.read_csv('NusaX_data/train.csv')
valid_data = pd.read_csv('NusaX_data/valid.csv')
test_data = pd.read_csv('NusaX_data/test.csv')

train_texts = train_data['text'].tolist()
train_labels = train_data['label'].tolist()

In [101]:
# split data
rnn_train_texts, rnn_test_texts, rnn_train_labels, rnn_test_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=42)

In [102]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
rnn_train_labels = label_encoder.fit_transform(rnn_train_labels)
rnn_test_labels = label_encoder.transform(rnn_test_labels)

vectorizer = layers.TextVectorization(max_tokens=10000, output_sequence_length=100)
vectorizer.adapt(rnn_train_texts)

rnn_train_tokens = vectorizer(rnn_train_texts).numpy()
rnn_test_tokens = vectorizer(rnn_test_texts).numpy()

print(f"Token shapes - Train: {rnn_train_tokens.shape}, Test: {rnn_test_tokens.shape}")

Token shapes - Train: (400, 100), Test: (100, 100)


In [103]:
def create_rnn_model(rnn_layers=[64], bidirectional=False, vocab_size=10000, embedding_dim=128):
    model = keras.Sequential()
    model.add(layers.Input(shape=(100,)))
    model.add(layers.Embedding(vocab_size, embedding_dim, mask_zero=True))

    for i, units in enumerate(rnn_layers):
        return_sequences = (i < len(rnn_layers) - 1)
        if bidirectional:
            model.add(layers.Bidirectional(
                layers.SimpleRNN(units, return_sequences=return_sequences, dropout=0.2)
            ))
        else:
            model.add(layers.SimpleRNN(units, return_sequences=return_sequences, dropout=0.2))
        model.add(layers.Dropout(0.3))
    model.add(layers.Dense(3, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [104]:
def train_rnn_model_fixed(model, epochs=15):
    history = model.fit(
        rnn_train_tokens, rnn_train_labels,  # Use TOKENS, not text
        validation_data=(rnn_test_tokens, rnn_test_labels),
        epochs=epochs, batch_size=64, verbose=1
    )
    
    pred = model.predict(rnn_test_tokens)
    pred_classes = np.argmax(pred, axis=1)
    f1 = f1_score(rnn_test_labels, pred_classes, average='macro')
    return history, f1

In [105]:
print("Testing: Jumlah Layer RNN")
rnn_layer_results = {}

for num_layers in [1, 2, 3]:
    print(f"\nTraining RNN dengan {num_layers} layers...")
    
    rnn_config = [64] * num_layers
    model = create_rnn_model(rnn_layers=rnn_config)
    history, f1 = train_rnn_model_fixed(model)
    
    rnn_layer_results[num_layers] = {'f1': f1, 'history': history}
    model.save(f'test_result/rnn/rnn_{num_layers}_layers.h5')
    print(f"F1-Score: {f1:.4f}")

Testing: Jumlah Layer RNN

Training RNN dengan 1 layers...
Epoch 1/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 63ms/step - accuracy: 0.4054 - loss: 1.0786 - val_accuracy: 0.4500 - val_loss: 1.0193
Epoch 2/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.5802 - loss: 0.8712 - val_accuracy: 0.5300 - val_loss: 0.9935
Epoch 3/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.8956 - loss: 0.6714 - val_accuracy: 0.6100 - val_loss: 0.9152
Epoch 4/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.9734 - loss: 0.4543 - val_accuracy: 0.5800 - val_loss: 0.8989
Epoch 5/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.9959 - loss: 0.2569 - val_accuracy: 0.6200 - val_loss: 0.8382
Epoch 6/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 1.0000 - loss: 0.1468 - val_accuracy: 0.5700 - val_loss: 0.9088
Epoch 7/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.9987 - loss: 0.0817 - val_accuracy: 0.5700 - val_loss: 0.9065
Epoch 8/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 1.000

F1-Score: 0.5604

Training RNN dengan 2 layers...
Epoch 1/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 97ms/step - accuracy: 0.3689 - loss: 1.1399 - val_accuracy: 0.5300 - val_loss: 0.9881
Epoch 2/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.5305 - loss: 0.9118 - val_accuracy: 0.4900 - val_loss: 0.9266
Epoch 3/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.7108 - loss: 0.7475 - val_accuracy: 0.5800 - val_loss: 0.8619
Epoch 4/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.8283 - loss: 0.5577 - val_accuracy: 0.5700 - val_loss: 0.9163
Epoch 5/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.9384 - loss: 0.2956 - val_accuracy: 0.6100 - val_loss: 0.8529
Epoch 6/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.9727 - loss: 0.1787 - val_accuracy: 0.6000 - val_loss: 0.8823
Epoch 7/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.9911 - loss: 0.0999 - val_accuracy: 0.5900 - val_loss: 0.9376
Epoch 8/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.9946 - loss:

F1-Score: 0.5768

Training RNN dengan 3 layers...
Epoch 1/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 107ms/step - accuracy: 0.3653 - loss: 1.2992 - val_accuracy: 0.4800 - val_loss: 1.0197
Epoch 2/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.4946 - loss: 1.1092 - val_accuracy: 0.5100 - val_loss: 0.9898
Epoch 3/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.6534 - loss: 0.7558 - val_accuracy: 0.5300 - val_loss: 0.9998
Epoch 4/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - accuracy: 0.7224 - loss: 0.6129 - val_accuracy: 0.5600 - val_loss: 1.0629
Epoch 5/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.8207 - loss: 0.4677 - val_accuracy: 0.5500 - val_loss: 1.1723
Epoch 6/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.8913 - loss: 0.3349 - val_accuracy: 0.5500 - val_loss: 1.1441
Epoch 7/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.9333 - loss: 0.2148 - val_accuracy: 0.5500 - val_loss: 1.2833
Epoch 8/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.9679 - loss

F1-Score: 0.4762


In [106]:
print("\nTesting: Jumlah Cell RNN")
rnn_cell_results = {}

for num_cells in [32, 64, 128]:
    print(f"\nTraining RNN dengan {num_cells} cells...")
    
    rnn_config = [num_cells, num_cells]
    model = create_rnn_model(rnn_layers=rnn_config)
    history, f1 = train_rnn_model_fixed(model)
    
    rnn_cell_results[num_cells] = {'f1': f1, 'history': history}
    model.save(f'test_result/rnn/rnn_{num_cells}_cells.h5')
    print(f"F1-Score: {f1:.4f}")


Testing: Jumlah Cell RNN

Training RNN dengan 32 cells...
Epoch 1/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - accuracy: 0.3476 - loss: 1.1578 - val_accuracy: 0.3000 - val_loss: 1.1572
Epoch 2/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.5005 - loss: 1.0076 - val_accuracy: 0.3500 - val_loss: 1.1276
Epoch 3/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.7038 - loss: 0.8264 - val_accuracy: 0.3400 - val_loss: 1.1229
Epoch 4/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.7901 - loss: 0.6817 - val_accuracy: 0.3700 - val_loss: 1.1296
Epoch 5/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.8829 - loss: 0.4913 - val_accuracy: 0.4000 - val_loss: 1.1567
Epoch 6/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.9616 - loss: 0.3351 - val_accuracy: 0.3900 - val_loss: 1.2459
Epoch 7/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.9855 - loss: 0.2139 - val_accuracy: 0.3900 - val_loss: 1.2788
Epoch 8/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.995

F1-Score: 0.3927

Training RNN dengan 64 cells...
Epoch 1/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - accuracy: 0.3673 - loss: 1.1633 - val_accuracy: 0.4200 - val_loss: 1.0477
Epoch 2/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.5556 - loss: 0.9152 - val_accuracy: 0.5500 - val_loss: 0.9530
Epoch 3/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.7264 - loss: 0.6890 - val_accuracy: 0.6100 - val_loss: 0.8887
Epoch 4/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.8799 - loss: 0.4457 - val_accuracy: 0.5900 - val_loss: 0.9255
Epoch 5/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.9659 - loss: 0.2234 - val_accuracy: 0.6100 - val_loss: 0.8954
Epoch 6/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.9855 - loss: 0.1342 - val_accuracy: 0.5700 - val_loss: 1.0298
Epoch 7/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.9946 - loss: 0.0776 - val_accuracy: 0.5700 - val_loss: 1.1001
Epoch 8/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.9987 - loss:

F1-Score: 0.5532

Training RNN dengan 128 cells...
Epoch 1/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 114ms/step - accuracy: 0.4186 - loss: 1.1094 - val_accuracy: 0.4200 - val_loss: 1.1436
Epoch 2/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - accuracy: 0.6083 - loss: 0.8093 - val_accuracy: 0.5200 - val_loss: 0.9840
Epoch 3/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - accuracy: 0.8680 - loss: 0.5141 - val_accuracy: 0.5400 - val_loss: 1.0935
Epoch 4/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 78ms/step - accuracy: 0.9607 - loss: 0.2465 - val_accuracy: 0.6000 - val_loss: 1.1287
Epoch 5/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - accuracy: 0.9933 - loss: 0.0884 - val_accuracy: 0.5900 - val_loss: 1.2290
Epoch 6/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - accuracy: 0.9916 - loss: 0.0568 - val_accuracy: 0.4900 - val_loss: 1.8991
Epoch 7/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - accuracy: 0.9920 - loss: 0.0380 - val_accuracy: 0.5200 - val_loss: 1.7137
Epoch 8/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step - accuracy: 0.9968 - los

F1-Score: 0.5450


In [107]:
print("\nTesting: Arah RNN")
rnn_direction_results = {}

for bidirectional in [False, True]:
    direction = "bidirectional" if bidirectional else "unidirectional"
    print(f"\nTraining {direction} RNN...")
    
    rnn_config = [64, 64]
    model = create_rnn_model(rnn_layers=rnn_config, bidirectional=bidirectional)
    history, f1 = train_rnn_model_fixed(model)
    
    rnn_direction_results[direction] = {'f1': f1, 'history': history}
    model.save(f'rnn_{direction}.h5')
    print(f"F1-Score: {f1:.4f}")


Testing: Arah RNN

Training unidirectional RNN...
Epoch 1/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 78ms/step - accuracy: 0.3646 - loss: 1.1516 - val_accuracy: 0.5100 - val_loss: 0.9965
Epoch 2/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.5693 - loss: 0.9418 - val_accuracy: 0.5600 - val_loss: 0.9322
Epoch 3/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.7117 - loss: 0.6908 - val_accuracy: 0.6300 - val_loss: 0.8311
Epoch 4/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.8593 - loss: 0.4585 - val_accuracy: 0.6500 - val_loss: 0.7992
Epoch 5/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.9686 - loss: 0.2735 - val_accuracy: 0.6400 - val_loss: 0.8339
Epoch 6/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.9929 - loss: 0.1368 - val_accuracy: 0.6800 - val_loss: 0.8345
Epoch 7/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.9852 - loss: 0.0833 - val_accuracy: 0.6600 - val_loss: 0.8675
Epoch 8/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 1.0000 - loss

F1-Score: 0.5942

Training bidirectional RNN...
Epoch 1/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 107ms/step - accuracy: 0.3851 - loss: 1.1335 - val_accuracy: 0.5000 - val_loss: 1.0237
Epoch 2/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.6074 - loss: 0.8373 - val_accuracy: 0.5500 - val_loss: 0.9622
Epoch 3/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.7951 - loss: 0.6076 - val_accuracy: 0.6100 - val_loss: 0.8847
Epoch 4/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.9380 - loss: 0.3368 - val_accuracy: 0.6400 - val_loss: 0.8208
Epoch 5/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.9815 - loss: 0.1793 - val_accuracy: 0.6200 - val_loss: 0.8697
Epoch 6/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.9913 - loss: 0.0912 - val_accuracy: 0.6300 - val_loss: 0.9003
Epoch 7/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 1.0000 - loss: 0.0479 - val_accuracy: 0.6000 - val_loss: 1.0314
Epoch 8/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.9975 - loss: 

F1-Score: 0.5245


In [108]:
print("\nTesting RNN From Scratch Implementation")
best_rnn_model = keras.models.load_model('rnn_bidirectional.h5')
rnn_scratch = RNNModelFromScratch(best_rnn_model)
print("Comparing RNN implementations...")
keras_probs = best_rnn_model.predict(rnn_test_tokens)
keras_preds = np.argmax(keras_probs, axis=1)
keras_f1 = f1_score(rnn_test_labels, keras_preds, average='macro')

print(f"Input tokens shape: {rnn_test_tokens.shape}")
scratch_probs = rnn_scratch.forward(rnn_test_tokens, training=False)
print(f"Keras probs shape: {keras_probs.shape}")
print(f"Scratch probs shape: {scratch_probs.shape}")

if len(scratch_probs.shape) == 3:
    scratch_probs = scratch_probs[:, -1, :]
    print(f"Adjusted scratch probs shape: {scratch_probs.shape}")

if scratch_probs.shape != keras_probs.shape:
    print(f"Warning: Shape mismatch! Keras: {keras_probs.shape}, Scratch: {scratch_probs.shape}")
    if len(scratch_probs.shape) == 1:
        scratch_probs = scratch_probs.reshape(-1, 1)
    elif scratch_probs.shape[0] != keras_probs.shape[0]:
        min_samples = min(scratch_probs.shape[0], keras_probs.shape[0])
        scratch_probs = scratch_probs[:min_samples]
        keras_probs = keras_probs[:min_samples]
        rnn_test_labels = rnn_test_labels[:min_samples]

scratch_preds = np.argmax(scratch_probs, axis=1)
print(f"Final shapes - True labels: {rnn_test_labels.shape}, Keras preds: {keras_preds.shape}, Scratch preds: {scratch_preds.shape}")

min_len = min(len(rnn_test_labels), len(keras_preds), len(scratch_preds))
rnn_test_labels_adj = rnn_test_labels[:min_len]
keras_preds_adj = keras_preds[:min_len]
scratch_preds_adj = scratch_preds[:min_len]

keras_f1 = f1_score(rnn_test_labels_adj, keras_preds_adj, average='macro')
scratch_f1 = f1_score(rnn_test_labels_adj, scratch_preds_adj, average='macro')

print(f"Keras RNN F1-Score: {keras_f1:.4f}")
print(f"Scratch RNN F1-Score: {scratch_f1:.4f}")
print(f"Difference: {abs(keras_f1 - scratch_f1):.4f}")

print(f"Sample predictions - Keras: {keras_preds_adj[:5]}, Scratch: {scratch_preds_adj[:5]}")
print(f"True labels: {rnn_test_labels_adj[:5]}")


Testing RNN From Scratch Implementation
Comparing RNN implementations...
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step
Input tokens shape: (100, 100)
Keras probs shape: (100, 3)
Scratch probs shape: (100, 100, 3)
Adjusted scratch probs shape: (100, 3)
Final shapes - True labels: (100,), Keras preds: (100,), Scratch preds: (100,)
Keras RNN F1-Score: 0.5245
Scratch RNN F1-Score: 0.1728
Difference: 0.3516
Sample predictions - Keras: [1 2 2 1 2], Scratch: [0 0 0 0 0]
True labels: [1 1 2 1 0]


In [109]:
# variasi jumlah layer
print("Testing: Jumlah Layer LSTM")
lstm_layer_results = {}

for num_layers in [1, 2, 3]:
    print(f"\nTraining LSTM dengan {num_layers} layers...")
    
    model = create_lstm_model(lstm_units=64, num_lstm_layers=num_layers, vocab_size=10000, num_classes=3)
    history = train_lstm_model(model, rnn_train_tokens,rnn_train_labels,rnn_test_tokens, rnn_test_labels, epochs=3)
    
    pred = model.predict(rnn_test_tokens)
    pred_classes = np.argmax(pred, axis=1)
    f1 = f1_score(rnn_test_labels, pred_classes, average='macro')
    
    lstm_layer_results[num_layers] = {'f1': f1, 'history': history}
    model.save(f'test_result/lstm/lstm_{num_layers}_layers.h5')
    print(f"F1-Score: {f1:.4f}")

Testing: Jumlah Layer LSTM

Training LSTM dengan 1 layers...
Epoch 1/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.4212 - loss: 1.0921 - val_accuracy: 0.6200 - val_loss: 1.0718
Epoch 2/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5004 - loss: 1.0418 - val_accuracy: 0.4300 - val_loss: 0.9783
Epoch 3/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5264 - loss: 0.9588 - val_accuracy: 0.6400 - val_loss: 0.8931
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


F1-Score: 0.5111

Training LSTM dengan 2 layers...
Epoch 1/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 84ms/step - accuracy: 0.3981 - loss: 1.0915 - val_accuracy: 0.3800 - val_loss: 1.0590
Epoch 2/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - accuracy: 0.4238 - loss: 1.0165 - val_accuracy: 0.4000 - val_loss: 1.0248
Epoch 3/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - accuracy: 0.6091 - loss: 0.8456 - val_accuracy: 0.6300 - val_loss: 0.8594
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step


F1-Score: 0.5062

Training LSTM dengan 3 layers...
Epoch 1/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 116ms/step - accuracy: 0.3881 - loss: 1.0909 - val_accuracy: 0.3800 - val_loss: 1.0263
Epoch 2/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step - accuracy: 0.4148 - loss: 0.9864 - val_accuracy: 0.6400 - val_loss: 0.9019
Epoch 3/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 97ms/step - accuracy: 0.6921 - loss: 0.7825 - val_accuracy: 0.6400 - val_loss: 0.8106
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step


F1-Score: 0.4949


In [110]:
# cvariasi jumlah sel
print("\nTesting: Jumlah Cell LSTM")
lstm_cell_results = {}

for num_cells in [32, 64, 128]:
    print(f"\nTraining LSTM dengan {num_cells} cells...")
    
    model = create_lstm_model(lstm_units=num_cells, num_lstm_layers=2, vocab_size=10000, num_classes=3)
    history = train_lstm_model(model, rnn_train_tokens, rnn_train_labels,
                              rnn_test_tokens, rnn_test_labels, epochs=3)
    
    pred = model.predict(rnn_test_tokens)
    pred_classes = np.argmax(pred, axis=1)
    f1 = f1_score(rnn_test_labels, pred_classes, average='macro')
    
    lstm_cell_results[num_cells] = {'f1': f1, 'history': history}
    model.save(f'test_result/lstm/lstm_{num_cells}_cells.h5')
    print(f"F1-Score: {f1:.4f}")


Testing: Jumlah Cell LSTM

Training LSTM dengan 32 cells...
Epoch 1/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - accuracy: 0.3500 - loss: 1.0942 - val_accuracy: 0.5700 - val_loss: 1.0743
Epoch 2/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.4561 - loss: 1.0504 - val_accuracy: 0.4300 - val_loss: 1.0084
Epoch 3/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.4957 - loss: 0.9557 - val_accuracy: 0.6600 - val_loss: 0.9348
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step


F1-Score: 0.5082

Training LSTM dengan 64 cells...
Epoch 1/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 83ms/step - accuracy: 0.3834 - loss: 1.0896 - val_accuracy: 0.4600 - val_loss: 1.0450
Epoch 2/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - accuracy: 0.4620 - loss: 0.9915 - val_accuracy: 0.5900 - val_loss: 1.0074
Epoch 3/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 0.5311 - loss: 0.9143 - val_accuracy: 0.6300 - val_loss: 0.8700
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step


F1-Score: 0.5210

Training LSTM dengan 128 cells...
Epoch 1/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 148ms/step - accuracy: 0.3635 - loss: 1.0875 - val_accuracy: 0.3800 - val_loss: 1.0170
Epoch 2/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 132ms/step - accuracy: 0.4421 - loss: 0.9734 - val_accuracy: 0.6000 - val_loss: 0.9218
Epoch 3/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 133ms/step - accuracy: 0.6415 - loss: 0.8300 - val_accuracy: 0.6400 - val_loss: 0.7661
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step


F1-Score: 0.4946


In [111]:
# pengaruh arah
print("\nTesting: Arah LSTM")
lstm_direction_results = {}

for bidirectional in [False, True]:
    direction = "bidirectional" if bidirectional else "unidirectional"
    print(f"\nTraining {direction} LSTM...")
    
    model = create_lstm_model(lstm_units=64, num_lstm_layers=2, 
                             bidirectional=bidirectional, vocab_size=10000, num_classes=3)
    history = train_lstm_model(model, rnn_train_tokens, rnn_train_labels,
                              rnn_test_tokens, rnn_test_labels, epochs=3)
    
    pred = model.predict(rnn_test_tokens)
    pred_classes = np.argmax(pred, axis=1)
    f1 = f1_score(rnn_test_labels, pred_classes, average='macro')
    
    lstm_direction_results[direction] = {'f1': f1, 'history': history}
    model.save(f'test_result/lstm/lstm_{direction}.h5')
    print(f"F1-Score: {f1:.4f}")


Testing: Arah LSTM

Training unidirectional LSTM...
Epoch 1/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 110ms/step - accuracy: 0.3867 - loss: 1.0946 - val_accuracy: 0.3500 - val_loss: 1.0802
Epoch 2/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - accuracy: 0.4248 - loss: 1.0470 - val_accuracy: 0.4400 - val_loss: 1.0340
Epoch 3/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - accuracy: 0.4471 - loss: 0.9588 - val_accuracy: 0.5500 - val_loss: 0.9878
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step


F1-Score: 0.4580

Training bidirectional LSTM...
Epoch 1/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 119ms/step - accuracy: 0.3716 - loss: 1.0909 - val_accuracy: 0.3900 - val_loss: 1.0612
Epoch 2/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - accuracy: 0.4168 - loss: 1.0251 - val_accuracy: 0.4500 - val_loss: 0.9910
Epoch 3/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - accuracy: 0.6173 - loss: 0.8537 - val_accuracy: 0.6800 - val_loss: 0.8401
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 123ms/step


F1-Score: 0.5769


In [112]:
print("\nTesting LSTM From Scratch Implementation")
best_lstm_model_path = 'test_result/lstm/lstm_bidirectional.h5'
best_lstm_model = keras.models.load_model(best_lstm_model_path)

print("Comparing LSTM Keras model performance...")
keras_probs = best_lstm_model.predict(rnn_test_tokens)
keras_preds = np.argmax(keras_probs, axis=1)
keras_f1 = f1_score(rnn_test_labels, keras_preds, average='macro')

print(f"Keras LSTM F1-Score: {keras_f1:.4f}")
print("Testing LSTM model consistency...")
batch1_probs = best_lstm_model.predict(rnn_test_tokens[:10])
batch2_probs = best_lstm_model.predict(rnn_test_tokens[10:20])

batch1_preds = np.argmax(batch1_probs, axis=1)
batch2_preds = np.argmax(batch2_probs, axis=1)

print(f"LSTM Batch 1 predictions: {batch1_preds}")
print(f"LSTM Batch 2 predictions: {batch2_preds}")

print(f"Model input shape: {best_lstm_model.input_shape}")
print(f"Model output shape: {best_lstm_model.output_shape}")
print(f"Number of parameters: {best_lstm_model.count_params():,}")


Testing LSTM From Scratch Implementation


Comparing LSTM Keras model performance...
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 119ms/step
Keras LSTM F1-Score: 0.5769
Testing LSTM model consistency...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
LSTM Batch 1 predictions: [0 2 2 0 0 0 2 2 0 2]
LSTM Batch 2 predictions: [2 2 2 0 2 0 0 2 0 0]
Model input shape: (None, 100)
Model output shape: (None, 3)
Number of parameters: 1,183,683
